# 1.0 Import Dependencies

In [1]:
import os, platform, time, urllib.request, openpyxl, operator
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from openpyxl import Workbook
import sys, requests, re, json
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import utils
from utils import scrape_username, scrape_general_info

# 2.0 Scrape Username from Star n Gage Website

In [13]:
#scrape username instagram that we want to analyze
for i in range(10): #bcs the number of webpage is until 10
    a = i+1
    #if response 200 is that webpage can be scraped
    url = f'https://starngage.com/app/id/influencer/ranking?page={a}'
    
    #call function
    ranking, username, category = scrape_username(url)

response <Response [200]>
response <Response [200]>
response <Response [200]>
response <Response [200]>
response <Response [200]>
response <Response [200]>
response <Response [200]>
response <Response [200]>
response <Response [200]>
response <Response [200]>


In [14]:
inf_dict = {
    'username':username,
    'ranking':ranking,
    'category' : category
}

In [15]:
#Assign to dataframe
df = pd.DataFrame(inf_dict)

#do some preprocessing
df['username_2'] = df['username'].apply(lambda x: x[1:])
df['category'] = df['category'].astype('str')
df_2 = df.drop_duplicates().reset_index(drop=True)

In [16]:
df_2

,username,ranking,category,username_2
0,@raffinagita1717,1,['Entertainment'],raffinagita1717
1,@princessyahrini,2,['Singer'],princessyahrini
2,@laudyacynthiabella,3,"['Music of Asia', 'Singer', 'Film, Music & Boo...",laudyacynthiabella
3,@gisel_la,4,"['Music of Asia', 'Singer']",gisel_la
4,@lunamaya,5,['Actors'],lunamaya
...,...,...,...,...
995,@receh.id,996,[],receh.id
996,@zigzagbatamnew,997,['Shopping'],zigzagbatamnew
997,@bioskop,998,['Comics'],bioskop
998,@hijrahfest,999,[],hijrahfest


In [6]:
# df_2.to_csv(r'influencer_names_raw.csv', index=False)

# 3.0 Scrape General Information
- Followers, Following, Jumlah Post

In [7]:
df_2['followers'] = 'Null'
df_2['following'] = 'Null'
df_2['post'] = 'Null'

In [8]:
#This process is take some time bcs we scrape 1000 influencers data
i = 0
for name in df_2['username_2']:
    # user name 
    url = f"https://www.instagram.com/{name}/"

    # calling scrape function
    try:
        data = scrape_general_info(url)
        df_2.loc[i, 'followers'] = data['Followers']
        df_2.loc[i, 'following'] = data['Following']
        df_2.loc[i, 'post'] = data['Posts']
        i += 1
    except:
        i += 1
        continue

# 4.0 Data Preprocessing

## 4.1 Feature Followers

In [69]:
df_3 = df_2[df_2['followers']!='Null'].reset_index(drop=True)

In [72]:
df_3['multiplier'] = df_3['followers'].apply(lambda x: x[-1])
df_3['followers'] = df_3['followers'].apply(lambda x: x[:-1])
df_3['followers_2'] = np.nan

In [88]:
for i in range(len(df_3)):
    try:
        df_3['followers_2'][i] = float(df_3['followers'][i])
    except:
        continue

C:\Users\80119075\AppData\Local\Continuum\anaconda3\envs\python-cpu\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [93]:
df_4 = df_3[df_3['followers_2'].notnull()].reset_index(drop=True)
df_5 = df_4[(df_4['multiplier']=='k') | (df_4['multiplier']=='m')].reset_index(drop=True)

In [108]:
df_5['multiplier'] = df_5['multiplier'].replace({
    'm' : 1000000,
    'k' : 1000
})

In [111]:
df_5['followers_3'] = (df_5['followers_2'] * df_5['multiplier']).astype('int')

## 4.2 Feature Following

In [195]:
df_6 = df_5[df_5['following']!='follow'].reset_index(drop=True)
df_6['following'] = df_6['following'].replace(r'.,', '', regex=True).astype('int')

## 4.3 Feature Post

In [197]:
df_6['post'] = df_6['post'].replace(r',', '', regex=True)
df_6['multiplier_2'] = df_6['post'].apply(lambda x: x[-1])
df_6['multiplier_2'] = df_6['multiplier_2'].apply(lambda x: 1 if x!='k' else 1000)

In [200]:
for i in range(len(df_6)):
    if df_6['multiplier_2'][i] == 1000:
        df_6['post'][i] = float(df_6['post'][i][:-1]) * df_6['multiplier_2'][i]
    else:
        continue

In [206]:
df_final = df_6.drop(columns=['ranking_beauty', 'category', 'followers', 'multiplier',
                   'followers_2', 'multiplier_2', 'username']).rename(columns={
    'username_2' : 'username',
    'followers_3' : 'followers'
})

In [211]:
df_final['post'] = df_final['post'].astype('int')

In [212]:
df_final.to_csv('influencer_names_final.csv', index=False)